<a href="https://colab.research.google.com/github/mimilazarova/dd2412_project_fixmatch_and_beyond/blob/main/src/FixMatch_mimi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install tensorflow-addons==0.11.1

     |████████████████████████████████| 1.1MB 10.5MB/s 
  Found existing installation: tensorflow-addons 0.8.3
    Uninstalling tensorflow-addons-0.8.3:
      Successfully uninstalled tensorflow-addons-0.8.3


In [8]:
# All imports here
import tensorflow_probability as tfp
import numpy as np
from PIL import Image, ImageOps, ImageEnhance, ImageFilter
import os
import tensorflow as tf
import logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 
logging.getLogger('tensorflow').disabled = True
import tensorflow_datasets as tfds
import tensorflow_addons as tfa
from tqdm import tqdm, tqdm_notebook
from matplotlib import pyplot as plt
import json

In [9]:
# this cell is for the wide ResNet
def regularized_padded_conv(*args, **kwargs):
    return tf.keras.layers.Conv2D(*args, **kwargs, padding='same', kernel_regularizer=_regularizer,
                                  kernel_initializer='he_normal', use_bias=False)


def BN_ReLU(x):
    x = tf.keras.layers.BatchNormalization()(x)
    return tf.keras.layers.ReLU()(x)


def shortcut(x, filters, stride, mode):
    if x.shape[-1] == filters:
        return x
    elif mode == 'B':
        return regularized_padded_conv(filters, 1, strides=stride)(x)
    elif mode == 'B_original':
        x = regularized_padded_conv(filters, 1, strides=stride)(x)
        return tf.keras.layers.BatchNormalization()(x)
    elif mode == 'A':
        return tf.pad(tf.keras.layers.MaxPool2D(1, stride)(x) if stride>1 else x,
                      paddings=[(0, 0), (0, 0), (0, 0), (0, filters - x.shape[-1])])
    else:
        raise KeyError("Parameter shortcut_type not recognized!")
    

def original_block(x, filters, stride=1, **kwargs):
    c1 = regularized_padded_conv(filters, 3, strides=stride)(x)
    c2 = regularized_padded_conv(filters, 3)(BN_ReLU(c1))
    c2 = tf.keras.layers.BatchNormalization()(c2)
    
    mode = 'B_original' if _shortcut_type == 'B' else _shortcut_type
    x = shortcut(x, filters, stride, mode=mode)
    return tf.keras.layers.ReLU()(x + c2)
    
    
def preactivation_block(x, filters, stride=1, preact_block=False):
    flow = BN_ReLU(x)
    if preact_block:
        x = flow
        
    c1 = regularized_padded_conv(filters, 3, strides=stride)(flow)
    if _dropout:
        c1 = tf.keras.layers.Dropout(_dropout)(c1)
        
    c2 = regularized_padded_conv(filters, 3)(BN_ReLU(c1))
    x = shortcut(x, filters, stride, mode=_shortcut_type)
    return x + c2


def bootleneck_block(x, filters, stride=1, preact_block=False):
    flow = BN_ReLU(x)
    if preact_block:
        x = flow
         
    c1 = regularized_padded_conv(filters//_bootleneck_width, 1)(flow)
    c2 = regularized_padded_conv(filters//_bootleneck_width, 3, strides=stride)(BN_ReLU(c1))
    c3 = regularized_padded_conv(filters, 1)(BN_ReLU(c2))
    x = shortcut(x, filters, stride, mode=_shortcut_type)
    return x + c3


def group_of_blocks(x, block_type, num_blocks, filters, stride, block_idx=0):
    global _preact_shortcuts
    preact_block = True if _preact_shortcuts or block_idx == 0 else False
    
    x = block_type(x, filters, stride, preact_block=preact_block)
    for i in range(num_blocks-1):
        x = block_type(x, filters)
    return x


def Resnet(input_shape, n_classes, l2_reg=1e-4, group_sizes=(2, 2, 2), features=(16, 32, 64), strides=(1, 2, 2),
           shortcut_type='B', block_type='preactivated', first_conv={"filters": 16, "kernel_size": 3, "strides": 1},
           dropout=0, cardinality=1, bootleneck_width=4, preact_shortcuts=True):
    
    global _regularizer, _shortcut_type, _preact_projection, _dropout, _cardinality, _bootleneck_width, _preact_shortcuts
    _bootleneck_width = bootleneck_width # used in ResNeXts and bootleneck blocks
    _regularizer = tf.keras.regularizers.l2(l2_reg)
    _shortcut_type = shortcut_type # used in blocks
    _cardinality = cardinality # used in ResNeXts
    _dropout = dropout # used in Wide ResNets
    _preact_shortcuts = preact_shortcuts
    
    block_types = {'preactivated': preactivation_block,
                   'bootleneck': bootleneck_block,
                   'original': original_block}
    
    selected_block = block_types[block_type]
    inputs = tf.keras.layers.Input(shape=input_shape)
    flow = regularized_padded_conv(**first_conv)(inputs)
    
    if block_type == 'original':
        flow = BN_ReLU(flow)
    
    for block_idx, (group_size, feature, stride) in enumerate(zip(group_sizes, features, strides)):
        flow = group_of_blocks(flow,
                               block_type=selected_block,
                               num_blocks=group_size,
                               block_idx=block_idx,
                               filters=feature,
                               stride=stride)
    
    if block_type != 'original':
        flow = BN_ReLU(flow)
    
    flow = tf.keras.layers.GlobalAveragePooling2D()(flow)
    outputs = tf.keras.layers.Dense(n_classes, kernel_regularizer=_regularizer)(flow)
    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model


def load_weights_func(model, model_name):
    try: model.load_weights(os.path.join('saved_models', model_name + '.tf'))
    except tf.errors.NotFoundError: print("No weights found for this model!")
    return model


def cifar_wide_resnet(N, K, block_type='preactivated', shortcut_type='B', dropout=0, l2_reg=2.5e-4):
    assert (N-4) % 6 == 0, "N-4 has to be divisible by 6"
    lpb = (N-4) // 6 # layers per block - since N is total number of convolutional layers in Wide ResNet
    model = Resnet(input_shape=(32, 32, 3), n_classes=10, l2_reg=l2_reg, group_sizes=(lpb, lpb, lpb), features=(16*K, 32*K, 64*K),
                   strides=(1, 2, 2), first_conv={"filters": 16, "kernel_size": 3, "strides": 1}, shortcut_type=shortcut_type,
                   block_type=block_type, dropout=dropout, preact_shortcuts=True)
    return model


def WRN_28_2(shortcut_type='B', load_weights=False, dropout=0, l2_reg=2.5e-4):
    model = cifar_wide_resnet(28, 2, 'preactivated', shortcut_type, dropout=dropout, l2_reg=l2_reg)
    if load_weights: model = load_weights_func(model, 'cifar_WRN_28_10')
    return model

In [10]:
# This cell is for CTAugment

class CTAugment:

  def __init__(self, n_classes, decay=0.99, threshold=0.80, depth=2, n_bins=17):
    self.decay = decay
    self.threshold = threshold
    self.depth = depth
    self.n_bins = n_bins
    self.n_classes = n_classes
    self.xforms = []
    # self.bins = [[]]
    # self.weights = [[]]

    self.AUG_DICT = {
        "autocontrast": {"f": self.autocontrast, "weight": [np.ones(self.n_bins)*1.0]},
        "blur": {"f": self.blur, "weight": [np.ones(self.n_bins)*1.0]},
        "brightness": {"f": self.brightness, "weight":[np.ones(self.n_bins)*1.0]},
        "color": {"f": self.color, "weight": [np.ones(self.n_bins)*1.0]},
        "contrast": {"f": self.contrast, "weight": [np.ones(self.n_bins)*1.0]},
        "cutout": {"f": self.cutout, "weight": [np.ones(self.n_bins)*1.0]},
        "equalize": {"f": self.equalize, "weight": [np.ones(self.n_bins)*1.0]},
        "invert": {"f": self.invert, "weight": [np.ones(self.n_bins)*1.0]},
        "identity": {"f": self.identity, "weight": [np.ones(self.n_bins)*1.0]},
        "posterize": {"f": self.posterize, "weight": [np.ones(self.n_bins)*1.0]},
        "rescale": {"f": self.rescale, "weight": [np.ones(self.n_bins)*1.0, np.ones(6)*1.0]},
        "rotate": {"f": self.rotate, "weight": [np.ones(self.n_bins)*1.0]},
        "sharpness": {"f": self.sharpness, "weight": [np.ones(self.n_bins)*1.0]},
        "shear_x": {"f": self.shear_x, "weight": [np.ones(self.n_bins)*1.0]},
        "shear_y": {"f": self.shear_y, "weight": [np.ones(self.n_bins)*1.0]},
        "smooth": {"f": self.smooth, "weight": [np.ones(self.n_bins)*1.0]},
        "solarize": {"f": self.solarize, "weight": [np.ones(self.n_bins)*1.0]},
        "translate_x": {"f": self.translate_x, "weight": [np.ones(self.n_bins)*1.0]},
        "translate_y": {"f": self.translate_y, "weight": [np.ones(self.n_bins)*1.0]}
    }
    self.N = len(self.AUG_DICT.keys())
    self.options = list(self.AUG_DICT.keys())

    self.batch_choices = []
    self.batch_bins = []

  def weight_to_p(self, weight):
        p = weight + (1 - self.decay)  # Avoid to have all zero.
        p = p / p.max()
        p[p < self.threshold] = 0
        return p/np.sum(p)

  def augment(self, x, uniform_bin_sampling=False):
    aug_x = Image.fromarray(np.uint8(x))#255*x))

    choices = [self.options[i] for i in np.random.choice(np.arange(self.N), self.depth, replace=False)]
    bins = []

    for k in range(self.depth):
        choice_key = choices[k]
        
        transformation = self.AUG_DICT[choice_key]["f"]
        # sample bins
        if uniform_bin_sampling:
          p = np.ones(self.N)/self.N
        else:
          w = self.AUG_DICT[choice_key]["weight"][0]
          p = self.weight_to_p(w)
        curr_bins = {}
        curr_bins["bin"] = np.random.choice(np.arange(self.n_bins), p=p)

        if choice_key=="rescale":
          if uniform_bin_sampling:
            p = np.ones(6)/6
          else:
            w = self.AUG_DICT[choice_key]["weight"][1]
            p = self.weight_to_p(w)
          curr_bins["bin2"] = np.random.choice(np.arange(6), p=p)

        aug_x = transformation(aug_x, **curr_bins)
        bins.append(curr_bins)

    return np.array(aug_x), choices, bins

  def augment_batch(self, batch):
    aug_batch = tf.identity(batch)

    #aug_batch = tf.map_fn(aug_batch, self.augment)
    batch_choices = []
    batch_bins = []
    
    if batch.ndim == 3:
      sample, choices, bins = self.augment(sample)
      batch_choices.append(choices)
      batch_bins.append(bins)
    elif batch.ndim == 4:
      for sample in aug_batch:
        sample, choices, bins = self.augment(sample)
        batch_choices.append(choices)
        batch_bins.append(bins)

    return aug_batch, batch_choices, batch_bins

  def update_weights(self, label, pred, choices, bins):
    omega = 1 - (1 / (2*self.n_classes)) * np.sum(tf.math.abs(label - pred))

    for k, choice in enumerate(choices):
      w = self.AUG_DICT[choice]["weight"][0]
      bin = bins[k]["bin"]
      self.AUG_DICT[choice]["weight"][0][bin] = self.decay * w[bin] + (1 - self.decay) * omega
      # print(self.AUG_DICT[choice]["weight"][0])
      if choices[k] == "rescale":
        w = self.AUG_DICT[choice]["weight"][1]
        bin = bins[k]["bin2"]
        self.AUG_DICT[choice]["weight"][1][bin] = self.decay * w[bin] + (1 - self.decay) * omega



  def update_weights_batch(self, labels, preds, choices, bins):
    [self.update_weights(l, p, c, b) for l, p, c, b in zip(labels, preds, choices, bins)]

  def get_param(self, r_min, r_max, bin):
      possible_value = np.linspace(r_min, r_max, self.n_bins)
      return possible_value[bin]

  def autocontrast(self, x, bin):
      param = self.get_param(0, 1, bin)
      return Image.blend(x, ImageOps.autocontrast(x), param)
  
  def blur(self, x, bin):
      param = self.get_param(0, 1, bin)
      return Image.blend(x, x.filter(ImageFilter.BLUR), param)
  
  def brightness(self, x, bin):
      param = self.get_param(0, 1, bin)
      return ImageEnhance.Brightness(x).enhance(0.1 + 1.9*param)

  def color(self, x, bin):
      param = self.get_param(0, 1, bin)
      return ImageEnhance.Color(x).enhance(0.1 + 1.9*param)

  def contrast(self, x, bin):
      param = self.get_param(0, 1, bin)
      return ImageEnhance.Contrast(x).enhance(0.1 + 1.9*param)

  def cutout(self, x, bin):
    """Taken directlly from FixMatch code"""
    level = self.get_param(0, 0.5, bin)

    size = 1 + int(level * min(x.size) * 0.499)
    img_height, img_width = x.size
    height_loc = np.random.randint(low=0, high=img_height)
    width_loc = np.random.randint(low=0, high=img_width)
    upper_coord = (max(0, height_loc - size // 2), max(0, width_loc - size // 2))
    lower_coord = (min(img_height, height_loc + size // 2), min(img_width, width_loc + size // 2))
    pixels = x.load()  # create the pixel map
    for i in range(upper_coord[0], lower_coord[0]):  # for every col:
        for j in range(upper_coord[1], lower_coord[1]):  # For every row
            pixels[i, j] = (127, 127, 127)  # set the color accordingly
    return x

  def equalize(self, x, bin):
      param = self.get_param(0, 1, bin)
      return Image.blend(x, ImageOps.equalize(x), param)

  def invert(self, x, bin):
      param = self.get_param(0, 1, bin)
      return Image.blend(x, ImageOps.invert(x), param)
  
  def identity(self, x, bin):
      return x

  def posterize(self, x, bin):
      param = int(self.get_param(0, 8, bin))
      return ImageOps.posterize(x, param)

  def rescale(self, x, bin, bin2):
      param = self.get_param(0.5, 1, bin)
      methods = (Image.ANTIALIAS, Image.BICUBIC, Image.BILINEAR, Image.BOX, Image.HAMMING, Image.NEAREST)
      method = methods[bin2]
      s = x.size
      scale = param*0.25
      crop = (scale * s[0], scale * s[1], s[0] * (1 - scale), s[1] * (1 - scale))
      return x.crop(crop).resize(x.size, method)

  def rotate(self, x, bin):
      param = self.get_param(-45, 45, bin)
      angle = int(np.round((2 * param - 1) * 45))
      return x.rotate(angle)

  def sharpness(self, x, bin):
      param = self.get_param(0, 1, bin)
      return ImageEnhance.Sharpness(x).enhance(0.1 + 1.9*param)

  def shear_x(self, x, bin):
      param = self.get_param(-0.3, 0.3, bin)
      shear = (2 * param - 1) * 0.3
      return x.transform(x.size, Image.AFFINE, (1, shear, 0, 0, 1, 0))

  def shear_y(self, x, bin):
      param = self.get_param(-0.3, 0.3, bin)
      shear = (2 * param - 1) * 0.3
      return x.transform(x.size, Image.AFFINE, (1, 0, 0, shear, 1, 0))

  def smooth(self, x, bin):
      param = self.get_param(0, 1, bin)
      return Image.blend(x, x.filter(ImageFilter.SMOOTH), param)

  def solarize(self, x, bin):
      param = self.get_param(0, 1, bin)
      th = int(param * 255.999)
      return ImageOps.solarize(x, th)

  def translate_x(self, x, bin):
      param = self.get_param(-0.3, 0.3, bin)
      delta = (2 * param - 1) * 0.3
      return x.transform(x.size, Image.AFFINE, (1, 0, delta, 0, 1, 0))

  def translate_y(self, x, bin):
      param = self.get_param(-0.3, 0.3, bin)
      delta = (2 * param - 1) * 0.3
      return x.transform(x.size, Image.AFFINE, (1, 0, 0, 0, 1, delta))

In [11]:
def training(model, ds, mean=None, std=None,
             lr_values=[0.01, 0.1, 0.01, 0.001],
             lr_boundaries=[400, 32000, 48000, 64000],
             val_interval=2000, log_interval=200,
             batch_size=128, nesterov=False):

    schedule = tf.keras.optimizers.schedules.PiecewiseConstantDecay(boundaries=lr_boundaries[:-1], values=lr_values)
    optimizer = tf.keras.optimizers.SGD(schedule, momentum=0.9, nesterov=nesterov)
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
    def train_prep(x, y):
        x = tf.cast(x, tf.float32)
        #x = tf.image.random_flip_left_right(x)
        #x = tf.image.pad_to_bounding_box(x, 4, 4, 40, 40) #must fix ds independent shape
        #x = tf.image.random_crop(x, (32, 32, 3))          #must fix ds independent shape
        if mean is not None and std is not None:
          x = (x - mean) / std
        return x, y

    def valid_prep(x, y):
        x = tf.cast(x, tf.float32)#/255.
        if mean is not None and std is not None:
          x = (x - mean) / std
        return x, y


    #ds['train'] = ds['train'].map(train_prep).shuffle(10000).repeat().batch(batch_size).prefetch(-1)
    ds['train'] = ds['train'].map(train_prep).batch(batch_size).prefetch(-1)
    ds['test'] = ds['test'].map(valid_prep).batch(batch_size*4).prefetch(-1)

    #runid = run_name + '_x' + str(np.random.randint(10000))
    #writer = tf.summary.create_file_writer(logdir + '/' + runid)
    accuracy = tf.metrics.SparseCategoricalAccuracy()
    cls_loss = tf.metrics.Mean()
    reg_loss = tf.metrics.Mean()
    
    #print(f"RUNID: {runid}")
    #tf.keras.utils.plot_model(model)#, os.path.join('saved_plots', runid + '.png'))

    def weak_transformation(x):
      x = tf.image.random_flip_left_right(x)
      max_shift = tf.cast(x.shape[1]*0.125, dtype=tf.dtypes.int32)
      shift = tf.random.uniform([x.shape[0], 2], minval=-max_shift, maxval=max_shift, dtype=tf.dtypes.int32)
      return tfa.image.translate(x, tf.cast(shift, tf.dtypes.float32))

    def pseudolabel(class_dist, threshold):
        argmax = tf.math.argmax(class_dist, axis=1)
        max_probs = tf.math.multiply(tf.one_hot(argmax, class_dist.shape[1]), tf.nn.softmax(class_dist))
        # print(max_probs)
        # print(max_probs > threshold)
        res = tf.cast(max_probs > threshold, max_probs.dtype)
        print(res)
        return res

   
    #@tf.function
    def step(x, y, training):
        with tf.GradientTape() as tape:            
            # unlabeled data
            x_wk = weak_transformation(x)
            outs_wk = model(x_wk, training)  # should this be training or not?
            weak_labels = pseudolabel(outs_wk, 0.8)
            # print(outs_wk)
            # weak_labels = threshold_gate(weak_labels, outs_wk, 0.7)
            

            cta = CTAugment(10)

            x_str, choices, bins = cta.augment_batch(x)

            outs_str = model(x_str, training)

            cta.update_weights_batch(weak_labels, outs_str, choices, bins)
            
            
            #unlabeled_loss = loss_fn(weak_labels, outs_str)
            
            # labeled data
            #outs = model(x, training)
            #labeled_loss = loss_fn(y, outs)

            #add losses together
            #loss = labeled_loss + lambda * unlabeled_loss

            r_loss = tf.add_n(model.losses)
            outs = model(x, training)
            c_loss = loss_fn(y, outs)
            loss = c_loss + r_loss

        if training:
            gradients = tape.gradient(loss, model.trainable_weights)
            optimizer.apply_gradients(zip(gradients, model.trainable_weights))

        accuracy(y, outs)
        cls_loss(c_loss)
        reg_loss(r_loss)
        

    training_step = 0
    best_validation_acc = 0
    epochs = lr_boundaries[-1] // val_interval

    
    for epoch in range(epochs):
        for x, y in tqdm(ds['train'].take(val_interval), desc=f'epoch {epoch+1}/{epochs}',
                         total=val_interval, ncols=100, ascii=True):

            training_step += 1
            step(x, y, training=True)

            if training_step % log_interval == 0:
                #with writer.as_default():
                    c_loss, r_loss, err = cls_loss.result(), reg_loss.result(), 1-accuracy.result()
                    print(f" c_loss: {c_loss:^6.3f} | r_loss: {r_loss:^6.3f} | err: {err:^6.3f}", end='\r')

                    tf.summary.scalar('train/error_rate', err, training_step)
                    tf.summary.scalar('train/classification_loss', c_loss, training_step)
                    tf.summary.scalar('train/regularization_loss', r_loss, training_step)
                    tf.summary.scalar('train/learnig_rate', optimizer._decayed_lr('float32'), training_step)
                    cls_loss.reset_states()
                    reg_loss.reset_states()
                    accuracy.reset_states()

        for x, y in ds['test']:
            step(x, y, training=False)

        #with writer.as_default(): TBULATE THE FOLLOWING WHEN UNCOMMENTING!
        tf.summary.scalar('test/classification_loss', cls_loss.result(), step=training_step)
        tf.summary.scalar('test/error_rate', 1-accuracy.result(), step=training_step)
            
        if accuracy.result() > best_validation_acc:
                best_validation_acc = accuracy.result()
                #model.save_weights(os.path.join('saved_models', runid + '.tf'))
                
        cls_loss.reset_states()
        accuracy.reset_states()
            
    
    
def cifar_error_test(model, tr_len=20, vd_len=2):

    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    optimizer = tf.keras.optimizers.SGD(0.01)

    ds = tfds.load('cifar10', as_supervised=True, in_memory=True)
    std = tf.reshape((0.2023, 0.1994, 0.2010), shape=(1, 1, 3))
    mean= tf.reshape((0.4914, 0.4822, 0.4465), shape=(1, 1, 3))
    
    def train_prep(x, y):
        x = tf.cast(x, tf.float32)#/255.
        x = tf.image.random_flip_left_right(x)
        x = tf.image.pad_to_bounding_box(x, 4, 4, 40, 40)
        x = tf.image.random_crop(x, (32, 32, 3))
        x = (x - mean) / std
        return x, y

    def valid_prep(x, y):
        x = tf.cast(x, tf.float32)#/255.
        x = (x - mean) / std
        return x, y

    ds['train'] = ds['train'].map(train_prep).batch(5).take(tr_len).prefetch(-1)
    ds['test'] = ds['test'].map(valid_prep).batch(5).take(vd_len).prefetch(-1)

    accuracy = tf.metrics.SparseCategoricalAccuracy()
    cls_loss = tf.metrics.Mean()
    reg_loss = tf.metrics.Mean()

    #@tf.function
    def step(x, y, training):
        with tf.GradientTape() as tape:
            r_loss = tf.add_n(model.losses)
            outs = model(x, training)
            c_loss = loss_fn(y, outs)
            loss = c_loss + r_loss

        if training:
            gradients = tape.gradient(loss, model.trainable_weights)
            optimizer.apply_gradients(zip(gradients, model.trainable_weights))

        accuracy(y, outs)
        cls_loss(c_loss)
        reg_loss(r_loss)
        
    training_step = 0
    for x, y in tqdm(ds['train'], desc=f'test', total=tr_len, ncols=100, ascii=True):

        training_step += 1
        step(x, y, training=True)
        c_loss, r_loss, err = cls_loss.result(), reg_loss.result(), 1-accuracy.result()
        print(f" c_loss: {c_loss:^6.3f} | r_loss: {r_loss:^6.3f} | err: {err:^6.3f}", end='\r')

    for x, y in ds['test']:
        step(x, y, training=False)

In [12]:
#ds = tfds.load('cifar10', as_supervised=True, in_memory=True)
ds = tfds.load('cifar10', as_supervised=True)
model = WRN_28_2()

# training(model, ds)

/usr/local/lib/python3.6/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incomplete9SI0T3/cifar10-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/cifar10/3.0.0.incomplete9SI0T3/cifar10-test.tfrecord


Dataset cifar10 downloaded and prepared to /root/tensorflow_datasets/cifar10/3.0.0. Subsequent calls will reuse this data.


In [59]:
def ParseFunction(serialized, image_shape=[32, 32, 3]):
    features = {'image': tf.io.FixedLenFeature([], tf.string),
                'label': tf.io.FixedLenFeature([], tf.int64)}

    parsed_example = tf.io.parse_single_example(serialized=serialized, features=features) 
    image = tf.image.decode_image(parsed_example['image'])
    image.set_shape(image_shape)
    image = tf.cast(image, tf.float32) * (2.0 / 255) - 1.0
    data = dict(image=image, label=parsed_example['label'])
    return data

def stl_ParseFunction(input):
  return ParseFunction(serialized=input, image_shape=[96, 96, 3])


In [34]:
dir_path = "/content/drive/My Drive/Colab Notebooks"

# ds = tfds.load(os.path.join(dir_path, 'cifar10.3@10-label.tfrecord'))
ds = tf.data.TFRecordDataset(os.path.join(dir_path, 'cifar10.3@10-label.tfrecord'))

for raw in ds.take(1):
  parsed = tf.train.Example.FromString(raw.numpy())
  # print(parsed)
  image = parsed.features.feature['image'].bytes_list.value

In [40]:
dataset = ds.map(ParseFunction)

# it = tf.compat.v1.data.make_one_shot_iterator(dataset) # Never used?
images = np.stack([x['image'] for x in dataset])
labels = np.stack([x['label'] for x in dataset])

print(labels)

[4 6 5 1 9 0 8 3 2 7]


In [16]:
with open(os.path.join(dir_path, 'cifar10.3@10-label.json'), 'r') as f:
    jsn=json.load(f)

jsn

{'distribution': [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
 'label': [856, 9838, 14160, 16471, 24609, 25808, 38589, 38976, 47309, 49335]}

In [60]:
ds = tf.data.TFRecordDataset(os.path.join(dir_path, 'cifar100.3@100-label.tfrecord'))

dataset = ds.map(ParseFunction)

# it = tf.compat.v1.data.make_one_shot_iterator(dataset) # Never used?
images = np.stack([x['image'] for x in dataset])
labels = np.stack([x['label'] for x in dataset])

print(labels)

[ 7 27  4 48 30 39 97 93 64 94 76 91 84  6 19  3 26 49 68 62 78 37 60 18
 47 43 75  0 16 58 23 77 63 71 45 17 40 67 80 22 28 41 66  1 81 83 11 20
  9 70 46 34 89 21 99 59  2 95 51 38 29 50 79 32 72 25 61 87 10 14 52 98
 36 56 53 74 55 31 65 13 54 73 88 42 69 44  8 15 12 35 96 86 33 92  5 24
 82 57 90 85]


In [61]:
ds = tf.data.TFRecordDataset(os.path.join(dir_path, 'svhn.3@10-label.tfrecord'))
dataset = ds.map(ParseFunction)

# it = tf.compat.v1.data.make_one_shot_iterator(dataset) # Never used?
images = np.stack([x['image'] for x in dataset])
labels = np.stack([x['label'] for x in dataset])

print(labels)

[5 6 0 9 8 1 4 3 7 2]


In [62]:
ds = tf.data.TFRecordDataset(os.path.join(dir_path, 'svhn_noextra.3@10-label.tfrecord'))
dataset = ds.map(ParseFunction)

# it = tf.compat.v1.data.make_one_shot_iterator(dataset) # Never used?
images = np.stack([x['image'] for x in dataset])
labels = np.stack([x['label'] for x in dataset])

print(labels)

[0 9 3 2 7 4 5 6 1 8]


In [63]:
ds = tf.data.TFRecordDataset(os.path.join(dir_path, 'stl10.3@1000-label.tfrecord'))
dataset = ds.map(stl_ParseFunction)

# it = tf.compat.v1.data.make_one_shot_iterator(dataset) # Never used?
images = np.stack([x['image'] for x in dataset])
labels = np.stack([x['label'] for x in dataset])

print(labels)

[1 7 8 7 6 2 0 1 1 1 9 7 8 5 2 5 0 2 6 0 0 5 0 4 4 2 4 3 3 7 2 9 8 8 7 2 9
 9 4 4 3 4 7 7 8 2 1 5 3 3 2 8 1 4 1 9 7 1 1 2 4 7 2 1 8 8 5 9 2 8 0 3 8 6
 5 1 9 5 1 0 8 8 5 7 6 1 2 2 5 5 2 8 5 0 3 6 7 7 4 2 4 4 2 8 2 0 8 6 1 1 6
 1 6 9 9 2 0 3 2 1 9 1 4 0 4 5 6 0 7 2 2 8 1 9 5 2 8 0 3 7 0 9 7 7 9 5 4 3
 0 6 6 8 3 7 7 1 2 3 3 9 7 1 0 2 5 6 3 2 7 3 6 9 6 1 5 8 9 7 8 2 3 3 2 5 8
 0 1 3 3 9 0 7 6 8 6 7 3 6 6 7 0 9 8 5 1 2 4 3 1 9 9 7 1 6 6 0 2 4 4 8 0 4
 6 0 0 7 7 6 2 8 8 0 1 9 2 3 3 1 6 0 7 8 4 6 3 3 7 4 6 0 4 1 6 3 3 9 3 8 3
 7 4 0 4 6 6 6 6 2 1 3 8 5 4 9 5 7 4 5 2 5 0 9 4 8 6 5 4 8 6 1 2 5 3 1 8 0
 8 5 6 5 7 5 2 3 6 5 4 5 8 3 4 9 6 5 0 3 0 8 0 7 9 7 8 0 0 2 6 7 8 6 5 3 0
 7 8 5 5 9 3 9 9 5 5 4 4 9 3 0 1 7 8 0 1 5 4 4 7 0 3 1 8 6 1 6 1 8 8 7 1 3
 4 8 6 7 7 7 3 9 4 4 0 9 2 8 9 3 5 1 3 5 2 0 2 8 0 0 7 9 3 9 8 6 3 3 1 3 5
 8 1 0 2 2 9 7 3 4 0 2 7 9 0 5 4 8 7 6 6 4 7 1 3 0 2 2 2 0 1 3 5 1 9 6 5 4
 1 9 5 5 0 4 9 0 9 7 8 8 5 2 1 4 2 2 4 4 1 1 2 8 4 3 3 0 2 4 2 1 6 6 0 2 7
 8 9 5 9 9 0 5 4 0 3 5 1 

In [48]:
cf100 = tfds.load("cifar100", as_supervised=True)

for ex in cf100['train'].take(1):
  print(ex)
  im, l = ex

print(l)

(<tf.Tensor: shape=(32, 32, 3), dtype=uint8, numpy=
array([[[151, 154, 145],
        [144, 147, 135],
        [141, 143, 134],
        ...,
        [ 44,  38,  39],
        [112, 100,  96],
        [145, 127, 120]],

       [[153, 156, 150],
        [141, 144, 135],
        [139, 142, 136],
        ...,
        [ 68,  59,  59],
        [131, 113, 106],
        [121, 101,  94]],

       [[163, 165, 161],
        [150, 153, 145],
        [147, 149, 144],
        ...,
        [ 75,  66,  60],
        [101,  86,  77],
        [ 83,  77,  73]],

       ...,

       [[150, 150, 162],
        [ 51,  52,  67],
        [ 46,  42,  56],
        ...,
        [ 60,  59,  48],
        [ 71,  73,  56],
        [ 58,  62,  44]],

       [[146, 158, 168],
        [ 92, 106, 116],
        [ 73,  77,  85],
        ...,
        [ 54,  55,  40],
        [ 59,  66,  41],
        [ 52,  56,  28]],

       [[ 92,  95, 105],
        [ 86,  90, 100],
        [ 77,  74,  82],
        ...,
        [ 59,  61,  44

In [50]:
svhn = tfds.load("svhn_cropped", as_supervised=True)

for ex in svhn['train'].take(1):
  print(ex)
  im, l = ex

print(l)

Shuffling and writing examples to /root/tensorflow_datasets/svhn_cropped/3.0.0.incompleteG82JPV/svhn_cropped-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/svhn_cropped/3.0.0.incompleteG82JPV/svhn_cropped-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/svhn_cropped/3.0.0.incompleteG82JPV/svhn_cropped-extra.tfrecord


Dataset svhn_cropped downloaded and prepared to /root/tensorflow_datasets/svhn_cropped/3.0.0. Subsequent calls will reuse this data.
(<tf.Tensor: shape=(32, 32, 3), dtype=uint8, numpy=
array([[[126, 138, 136],
        [128, 138, 137],
        [129, 139, 138],
        ...,
        [ 19,  19,  19],
        [ 19,  21,  20],
        [ 19,  21,  20]],

       [[127, 139, 137],
        [127, 139, 137],
        [129, 140, 138],
        ...,
        [ 19,  21,  20],
        [ 19,  21,  20],
        [ 17,  21,  20]],

       [[128, 140, 139],
        [127, 139, 138],
        [128, 138, 137],
        ...,
        [ 19,  21,  20],
        [ 20,  22,  21],
        [ 17,  21,  20]],

       ...,

       [[ 91,  89,  96],
        [ 76,  75,  83],
        [ 69,  68,  77],
        ...,
        [ 94,  96,  91],
        [ 68,  70,  65],
        [ 43,  45,  42]],

       [[ 81,  78,  87],
        [ 70,  68,  78],
        [ 66,  65,  76],
        ...,
        [ 99, 100,  95],
        [ 73,  75,  70],
    